<a href="https://colab.research.google.com/github/RafaSFernandes/DS_Projects/blob/main/M09_2_Estudos_sobre_Regress%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Escola DNC

Aluno: Rafael Souza Fernandes

Material de estudo

# Materia 09 (p.2)
Modelos de Classificação e Regressão

#Bibliotecas

In [88]:
import pandas as pd
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

#1. Importando Dados

In [66]:
#montagem do drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
#caminho da pasta do arquivo
path = "/content/drive/MyDrive/Datasets_Modulos/Materia_09/"
filename = "US_Accidents.csv" #nome do arquivo

In [68]:
#leitura do arquivo csv
df = pd.read_csv(path + filename)

/tmp/ipython-input-68-1088554273.py:2: DtypeWarning: Columns (30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path + filename)


#2. Tratamento (simplificado)

In [69]:
#ajuste da coluna "Star_Time" para o tipo "datetime"
df["Start_Time"] = pd.to_datetime(df["Start_Time"], format="%d-%m-%Y %H:%M", errors="coerce")

In [70]:
#filtro de ocorrências de acidentes no ano de 2017
df = df[df["Start_Time"].dt.year.isin([2017])]

In [71]:
#classificação de nível de "High_Severity" para casos com "Severity" maior ou igual a 3
df["High_Severity"] = df["Severity"].apply(lambda x: 1 if x >= 3 else 0)

In [72]:
#verificando as dimensões do dataframe
df.shape

(108008, 48)

In [73]:
#verificando as primeiras linhas
df.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;,High_Severity
15249,A-2731849,2.0,2017-01-01 00:44:00,01-01-2017 06:57,41.65280,-93.69785,41.649353,-93.697770,0.238,Between I-35/I-80 and US-6/Douglas Ave - Accid...,...,False,False,False,True,False,Night,Night,Night,Night;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;,0
15250,A-2731850,2.0,2017-01-01 01:07:00,01-01-2017 07:07,30.02012,-95.82565,30.019387,-95.823781,0.123,Between Beginning of US-290 Freeway and Katy H...,...,False,False,False,False,False,Night,Night,Night,Night;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;,0
15251,A-2731851,2.0,2017-01-01 01:07:00,01-01-2017 07:07,30.02509,-95.83735,30.020120,-95.825650,0.780,At Badtke Rd - Accident.,...,False,False,False,False,False,Night,Night,Night,Night;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;,0
15252,A-2731852,3.0,2017-01-01 01:58:00,01-01-2017 07:58,32.78833,-97.05835,32.809220,-97.054290,1.462,Between Post and Paddock Rd and Riverside Pky ...,...,False,False,False,False,False,Night,Night,Night,Night;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;,1
15253,A-2731853,3.0,2017-01-01 02:00:00,01-01-2017 08:00,32.76268,-96.77526,32.724300,-96.762260,2.757,Between Lamar St/Exit 283 and Overton Rd/Exit ...,...,False,False,False,False,False,Night,Night,Night,Night;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;,1


In [74]:
#verificando a proporção de casos de "High_Severity" no dataframe
df["High_Severity"].value_counts(normalize=True)

,proportion
High_Severity,
0,0.692745
1,0.307255


In [75]:
#listando as colunas do df
df.columns

Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number', 'Street',
       'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;',
       'High_Severity'],
      dtype='object')

In [76]:
#variáveis independentes (qual delas influenciou na ocorrência de acidente de severidade alta?)
features = [
    "Start_Lat",
    "Start_Lng",
    "End_Lat",
    "End_Lng",
    "Distance(mi)",
    "Temperature(F)",
    "Wind_Chill(F)",
    "Humidity(%)",
    "Pressure(in)",
    "Visibility(mi)",
    "Wind_Speed(mph)",
    "Precipitation(in)"
]

In [77]:
#definindo a variável "target"
target = df.High_Severity

In [78]:
#verificando dados nulos
df[features].isnull().sum()

,0
Start_Lat,0
Start_Lng,0
End_Lat,0
End_Lng,0
Distance(mi),0
Temperature(F),2262
Wind_Chill(F),88283
Humidity(%),2375
Pressure(in),1802
Visibility(mi),2628


In [79]:
#preenchendo dados nulos com o valor zero
df[features] = df[features].fillna(0)

In [80]:
#verificando dados nulos em "target"
target.isnull().sum()

np.int64(0)

In [81]:
#definido X
X = df[features]

In [82]:
#definido o y
y = target

#3. ML

##3.1. Modelo inicial

In [84]:
#separação de variáveis de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [86]:
#criando o modelo Random Forest
model = RandomForestClassifier(random_state=42, n_jobs=-1)

#realizando o treinamento com as variáveis de treino
model.fit(X_train, y_train)

#prevendo dados nas variáveis de teste
y_pred = model.predict(X_test)

#avaliando o modelo
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.91      0.86     22530
           1       0.71      0.51      0.60      9873

    accuracy                           0.79     32403
   macro avg       0.76      0.71      0.73     32403
weighted avg       0.78      0.79      0.78     32403



##3.2. Modelo (com Oversampling)

In [117]:
#criando o oversampler
oversampler = RandomOverSampler(random_state=42)

#aplicando o oversampling (equilibra os dados, aumentando a quantidade de dados da classe minoritária)
X_train_over, y_train_over = oversampler.fit_resample(X_train, y_train)

In [118]:
#comparando a quantidade de dados (aumento nos dados da classe minoritária)
y_train_over.value_counts() - y_train.value_counts()

,count
High_Severity,
0,0
1,28979


In [119]:
#constatando a proporção dos dados
y_train_over.value_counts(normalize=True)

,proportion
High_Severity,
1,0.5
0,0.5


In [120]:
#constatando o aumento de dados da classe minoritária comparado aos dados iniciais
print(f"Dados do y_train inicial: {y_train.shape} \nDados do y_train oversampled: {y_train_over.shape}")

Dados do y_train inicial: (75605,) 
Dados do y_train oversampled: (104584,)


In [121]:
#realizando o treinamento com as variáveis de treino
model.fit(X_train_over, y_train_over)

#prevendo dados nas variáveis de teste
y_pred_over = model.predict(X_test)

#avaliando o modelo
print(classification_report(y_test, y_pred_over))

              precision    recall  f1-score   support

           0       0.83      0.87      0.85     22530
           1       0.66      0.58      0.62      9873

    accuracy                           0.78     32403
   macro avg       0.74      0.73      0.73     32403
weighted avg       0.78      0.78      0.78     32403



##3.3. Modelo (com Undersampling)

In [122]:
#criando o underrsampler
undersampler = RandomUnderSampler(random_state=42)

#aplicando o underrsampling (identifica as classes minoritárias e equilibra o dados)
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)

In [124]:
#comparando a quantidade de dados (redução nos dados da classe majoritária)
y_train_under.value_counts() - y_train.value_counts()

,count
High_Severity,
0,-28979
1,0


In [125]:
#constatando a proporção dos dados
y_train_under.value_counts(normalize=True)

,proportion
High_Severity,
0,0.5
1,0.5


In [126]:
#constatando a redução de dados da classe majoritária comparado aos dados iniciais
print(f"Dados do y_train inicial: {y_train.shape} \nDados do y_train undersampled: {y_train_under.shape}")

Dados do y_train inicial: (75605,) 
Dados do y_train undersampled: (46626,)


In [123]:
#realizando o treinamento com as variáveis de treino
model.fit(X_train_under, y_train_under)

#prevendo dados nas variáveis de teste
y_pred_under = model.predict(X_test)

#avaliando o modelo
print(classification_report(y_test, y_pred_under))

              precision    recall  f1-score   support

           0       0.86      0.71      0.78     22530
           1       0.53      0.75      0.62      9873

    accuracy                           0.72     32403
   macro avg       0.70      0.73      0.70     32403
weighted avg       0.76      0.72      0.73     32403



##3.4. SMOTE

Synthetic Minority Oversampling Technique